<a href="https://colab.research.google.com/github/Sikandarh11/CodeClause-Data-Science-Internship-/blob/main/CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d rizwan123456789/potato-disease-leaf-datasetpl


403 - Forbidden - Permission 'datasets.get' was denied


In [4]:
!chmod 600 /root/.kaggle/kaggle.json

In [10]:
import zipfile
zip_ref = zipfile.ZipFile('/content/potato-disease-leaf-datasetpld.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Path to dataset directory
train_dir = '/content/PLD_3_Classes_256/Training'
validation_dir = '/content/PLD_3_Classes_256/Testing'

# Define data augmentation parameters
data_augmentation = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training dataset with data augmentation and preprocessing
train_ds_augmented = data_augmentation.flow_from_directory(
    directory=train_dir,
    target_size=(256, 256),
    batch_size=8,
    class_mode='categorical'
)

# Load validation dataset
validation_ds_aug = data_augmentation.flow_from_directory(
    directory=validation_dir,
    target_size=(256, 256),
    batch_size=8,
    class_mode='categorical'
)

# Define a simpler model architecture
model = keras.Sequential([
    Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(256, 256, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'),
    Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'),
    Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'),
    Flatten(),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

# Compile the model with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=['accuracy'])

# Train the model
history = model.fit(
    train_ds_augmented,
    steps_per_epoch=train_ds_augmented.samples // 8,
    validation_data=validation_ds_aug,
    validation_steps=validation_ds_aug.samples // 8,
    epochs=10
)


Found 3251 images belonging to 3 classes.
Found 405 images belonging to 3 classes.
Epoch 1/10
406/406 [==============================] - 747s 2s/step - loss: 0.9998 - accuracy: 0.5597 - val_loss: 1.6906 - val_accuracy: 0.2575
Epoch 2/10
406/406 [==============================] - 744s 2s/step - loss: 0.8464 - accuracy: 0.6405 - val_loss: 0.6966 - val_accuracy: 0.7125
Epoch 3/10
406/406 [==============================] - 741s 2s/step - loss: 0.7592 - accuracy: 0.6809 - val_loss: 0.6420 - val_accuracy: 0.7375
Epoch 4/10
406/406 [==============================] - 729s 2s/step - loss: 0.6830 - accuracy: 0.7145 - val_loss: 0.5799 - val_accuracy: 0.7575
Epoch 5/10
406/406 [==============================] - 745s 2s/step - loss: 0.6640 - accuracy: 0.7274 - val_loss: 0.6435 - val_accuracy: 0.7200
Epoch 6/10
406/406 [==============================] - 745s 2s/step - loss: 0.6168 - accuracy: 0.7478 - val_loss: 0.5526 - val_accuracy: 0.7800
Epoch 7/10
406/406 [==============================] - 743s 

In [12]:
# Save the model
model.save("cnn_model_optimized.h5")